<a href="https://colab.research.google.com/github/Speedbird45Bravo/Keras_projects_21/blob/main/Keras_41221.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
from keras.layers import Dense
from keras.models import Sequential
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

df = pd.read_csv("https://projects.fivethirtyeight.com/soccer-api/club/spi_matches.csv").dropna()
df['total_goals'] = df['score1'] + df['score2']

In [22]:
total_goals = []

for g in df['total_goals']:
  if g > 0:
    total_goals.append("GOALS")
  else:
    total_goals.append("SCORELESS")

df['goal_label'] = total_goals

In [23]:
def load_model():
  model = Sequential()
  model.add(Dense(16, activation='relu'))
  model.add(Dense(2, activation='softmax'))
  model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
  return model

def to_cat(array):
  array = to_categorical(np.array(pd.factorize(array)[0]))
  return array

In [24]:
X = df[['score2','score1']].reset_index(drop=True)
y = df[['goal_label']].copy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=6)

In [26]:
model = load_model()
y_train = to_cat(y_train['goal_label'])
y_test = to_cat(y_test['goal_label'])

In [27]:
model.fit(X_train, y_train, epochs=40, batch_size=128, verbose=2, validation_split=0.2)

Epoch 1/40
101/101 - 1s - loss: 0.7959 - accuracy: 0.2896 - val_loss: 0.6017 - val_accuracy: 0.9275
Epoch 2/40
101/101 - 0s - loss: 0.4685 - accuracy: 0.9310 - val_loss: 0.3572 - val_accuracy: 0.9287
Epoch 3/40
101/101 - 0s - loss: 0.2880 - accuracy: 0.9310 - val_loss: 0.2354 - val_accuracy: 0.9287
Epoch 4/40
101/101 - 0s - loss: 0.2028 - accuracy: 0.9310 - val_loss: 0.1799 - val_accuracy: 0.9287
Epoch 5/40
101/101 - 0s - loss: 0.1630 - accuracy: 0.9310 - val_loss: 0.1532 - val_accuracy: 0.9287
Epoch 6/40
101/101 - 0s - loss: 0.1431 - accuracy: 0.9310 - val_loss: 0.1387 - val_accuracy: 0.9287
Epoch 7/40
101/101 - 0s - loss: 0.1313 - accuracy: 0.9310 - val_loss: 0.1292 - val_accuracy: 0.9287
Epoch 8/40
101/101 - 0s - loss: 0.1228 - accuracy: 0.9310 - val_loss: 0.1214 - val_accuracy: 0.9287
Epoch 9/40
101/101 - 0s - loss: 0.1155 - accuracy: 0.9310 - val_loss: 0.1144 - val_accuracy: 0.9287
Epoch 10/40
101/101 - 0s - loss: 0.1086 - accuracy: 0.9310 - val_loss: 0.1075 - val_accuracy: 0.9287

In [28]:
predictions = model.predict(X_test).round(0)
test, accuracy = model.evaluate(y_test, predictions)

126/126 [==============================] - 0s 1ms/step - loss: 0.3904 - accuracy: 0.9268


In [29]:
print("Test Accuracy: %.2f" % (accuracy * 100) + "%")

Test Accuracy: 92.68%
